In [1]:
import os
import functions as fun
from geopy import distance
from folium import Choropleth, Circle, Marker, Icon, Map
from folium.plugins import HeatMap, MarkerCluster
from dotenv import load_dotenv
import pandas as pd

In [2]:
San_Francisco = 37.7871253,-122.4015358

In [3]:
sub_SF = pd.read_csv("../Data/sub_SF.csv")


# SAN FRANCISCO

In [4]:
load_dotenv()

True

In [5]:
token_foursquare = os.getenv("Token_foursquare")

- First we look for Starbucks close to the office and clean the results keeping only the columns that we are interested about 

In [6]:
starbucks_SF = fun.get_results("starbucks", San_Francisco, 10, 1000, token_foursquare)

In [7]:
df_SF = fun.clean_results_first(starbucks_SF, "Coffee")

- We look for preschools or kindergarten around the office and clean the results keeping only the columns that we are interested about 

In [8]:
kinder_SF = fun.get_results("preschool", San_Francisco, 10, 3000, token_foursquare)

In [9]:
df_SF = fun.clean_results(kinder_SF, df_SF, "Preschool")

- We look for nightclubs around the office and clean the results keeping only the columns that we are interested about

In [10]:
club_SF = fun.get_results("night club", San_Francisco, 10, 3000, token_foursquare)

In [11]:
df_SF = fun.clean_results(club_SF, df_SF, "Club")


- We look for vegan restaurants around the office and clean the results keeping only the columns that we are interested about

In [12]:
vegan_SF = fun.get_results("vegan", San_Francisco, 10, 3000, token_foursquare)

In [13]:
df_SF = fun.clean_results(vegan_SF, df_SF, "Vegan restaurant")


- We look for basketball arena (Chase Center) around the office and clean the results keeping only the columns that we are interested about

In [14]:
Basketball_SF = fun.get_results("Chase center", San_Francisco, 1, 5000, token_foursquare)
df_SF = fun.clean_results(Basketball_SF, df_SF, "Basketball arena")

- We look for dog hairdressers around the office and clean the results keeping only the columns that we are interested about

In [15]:
pet_SF = fun.get_results("pet grooming", San_Francisco, 5, 3000, token_foursquare)
df_SF = fun.clean_results(pet_SF, df_SF, "Pet hairdresser")

- We look for the closest Airport to office and clean the results keeping only the columns that we are interested about

In [16]:
Airports_SF = fun.get_results_airport("Airport", San_Francisco, 2, 50000, token_foursquare)
df_SF = fun.clean_results_lo(Airports_SF, df_SF, "Airport")

- We add the data of the new office that we have selected

In [17]:
df_SF = fun.add_company(df_SF, "662 Mission St, San Francisco, CA 94105", "Soma", 37.7871253, -122.4015358)

- We create a new row with the distance from the office to the diferent places

In [18]:
df_SF["Distance"] = df_SF.apply(fun.calc_distance_SF, axis = 1).round(2)

In [19]:
df_SF

,name,address,neighborhood,lat,lon,category,Distance
0,Company,"662 Mission St, San Francisco, CA 94105",Soma,37.787125,-122.401536,NaN,0.00
1,Starbucks,"299 Fremont St (Folsom St), San Francisco, CA ...",Chinatown,37.788371,-122.393555,Coffee,716.46
2,Starbucks,"789 Mission St (at 4th St), San Francisco, CA ...",Civic Center,37.784470,-122.404008,Coffee,366.45
3,Starbucks,"865 Market St (5th street), San Francisco, CA ...",Civic Center,37.784144,-122.407429,Coffee,615.61
4,Starbucks,"7 Drumm St (California), San Francisco, CA 94111",Chinatown,37.793818,-122.396678,Coffee,857.26
5,Starbucks,"780 Market St (at 4th St), San Francisco, CA 9...",Civic Center,37.786145,-122.405612,Coffee,375.18
6,Starbucks,"462 Powell St (at Sutter St), San Francisco, C...",Chinatown,37.789098,-122.408438,Coffee,646.20
7,Starbucks,"390 Stockton St (at Sutter St), San Francisco,...",Chinatown,37.789293,-122.406729,Coffee,516.86
8,Starbucks,"170 Ofarrell St, San Francisco, CA 94102",Civic Center,37.786534,-122.408216,Coffee,592.08
9,Starbucks,"264 Kearny St (at Bush St), San Francisco, CA ...",Chinatown,37.790652,-122.403869,Coffee,442.11


In [20]:
SF_map = Map(location = San_Francisco, zoom_start = 12)

In [21]:
for index, row in df_SF.iterrows():
    
    #1. MARKER without icon
    district = {"location": [row["lat"], row["lon"]], "tooltip": row["name"]}
    
    #2. Icon
    if row["category"] == "Coffee":        
        icon = Icon (
            color="white",
            opacity = 0.6,
            prefix = "fa",
            icon="coffee",
            icon_color = "black"
        )
    elif row["category"] == "Preschool":
        icon = Icon (
            color="pink",
            opacity = 0.6,
            prefix = "fa",
            icon="graduation-cap",
            icon_color = "yellow"
        )
    elif row["category"] == "Club":
        icon = Icon (
            color="darkpurple",
            opacity = 0.6,
            prefix = "fa",
            icon="glass",
            icon_color = "white"
        )
    elif row["category"] == "Vegan restaurant":
        icon = Icon (
            color="green",
            opacity = 0.6,
            prefix = "fa",
            icon="cutlery",
            icon_color = "white"
        )
    elif row["category"] == "Basketball arena":
        icon = Icon (
            color="orange",
            opacity = 0.6,
            prefix = "fa",
            icon="futbol-o",
            icon_color = "white"
        )
    elif row["category"] == "Airport":
        icon = Icon (
            color="blue",
            opacity = 0.6,
            prefix = "fa",
            icon="plane",
            icon_color = "white"
            
        )
    elif row["category"] == "Pet hairdresser":
        icon = Icon (
            color="red",
            opacity = 0.6,
            prefix = "fa",
            icon="scissors",
            icon_color = "white"
        )
    else:
        row["category"] == "Company",
        icon = Icon (
        color="black",
        opacity = 0.9,
        prefix = "fa",
        icon = "briefcase",
        icon_color = "white",
        icon_size=(30, 30)
        )
            
    #3. Marker
    marker = Marker(**district, icon = icon, radius = 2)
    
    #4. Add the Marker
    marker.add_to(SF_map)

In [22]:
for index, row in sub_SF.iterrows():
    
    #1. MARKER without icon
    district = {"location": [row["Lat"], row["Long"]], "tooltip": row["name"]}
    
    #2. Icon       
    icon = Icon (
    color="white",
    opacity = 0.6,
    prefix = "fa",
    icon="building",
    icon_color = "black"
        )

            
    #3. Marker
    marker = Marker(**district, icon = icon, radius = 2)
    
    #4. Add the Marker
    marker.add_to(SF_map)

In [25]:
SF_map

In [24]:
SF_map.save('../images/SanFrancisco_map.html')